# Logistic Regression Project Scenario

### Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They have noticed that they have quite bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name: Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1 = Account manager assigned
    Years: Total Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

Once you have creted the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they dont have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the dataa

In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLib

We will use the numerical coluns. We will include Account Manager because it is easy enough, but keep in mind it probably wont be any sort of a signal because the agency mentioned its randomly assigned!

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
                                       'Total_Purchase',
                                       'Account_Manager',
                                       'Years',
                                       'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
final_data = output.select('features','churn')

## Test Traing Split

In [11]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr_churn = LogisticRegression(labelCol='churn')

In [14]:
fitted_churn_model = lr_churn.fit(train_churn)

In [15]:
training_sum = fitted_churn_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                649|                649|
|   mean| 0.1694915254237288|0.14021571648690292|
| stddev|0.37547486847601813| 0.3474785049744481|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let us evaluate the results on the data set we were given

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [19]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,9090.43,1.0...|    0|[1.66672520967217...|[0.84113871804784...|       0.0|
|[28.0,11128.95,1....|    0|[4.44891792018249...|[0.98844389392225...|       0.0|
|[28.0,11245.38,0....|    0|[4.10367634864245...|[0.98375635251225...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.16223229851196...|[0.99430373651238...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.82498194672600...|[0.97864706658323...|       0.0|
|[30.0,10744.14,1....|    1|[1.85580386966033...|[0.86480710477195...|       0.0|
|[30.0,11575.37,1....|    1|[4.21339569754115...|[0.98541969077951...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.72400644842654...|[0.97643179741304...|       0.0|
|[31.0,8688.21,0.0...|    0|[7.08570041415892...|[0.99916371162017...|       0.0|
|[32.0,8617.98,1

### Using AUC


In [22]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [23]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [25]:
auc

0.731042654028436

https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve check out for what is good AUC value?

### Predict on brand new unlabeled data

In [26]:
final_lr_mpdel = lr_churn.fit(final_data)

In [27]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [28]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [29]:
test_new_customers = assembler.transform(new_customers)

In [30]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [32]:
final_results = final_lr_mpdel.transform(test_new_customers)

In [33]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

